In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc

In [ ]:
# Selected columns
program_cols = [
    'UNITID', 'INSTNM', 'CIPCODE', 'CIPDESC', 'CREDLEV', 'CREDDESC',
    'EARN_MDN_4YR', 'EARN_MDN_1YR', 'EARN_COUNT_WNE_4YR', 'EARN_COUNT_WNE_1YR',
    'DISTANCE'
]

institution_cols = [
    'UNITID', 'MD_EARN_WNE_P10', 'STABBR', 'REGION', 'CONTROL',
    'COSTT4_A', 'TUITIONFEE_IN', 'C150_4', 'ADM_RATE', 'PCTPELL'
]

# Load programs
df_programs = pd.read_csv(
    '/content/drive/MyDrive/5450_Files/Most-Recent-Cohorts-Field-of-Study.csv',
    usecols=program_cols,
    low_memory=False
)
print(f"Programs loaded: {len(df_programs):,}")

# Load institutions
df_institutions = pd.read_csv(
    '/content/drive/MyDrive/5450_Files/Most-Recent-Cohorts-Institution.csv',
    usecols=institution_cols,
    low_memory=False
).drop_duplicates(subset='UNITID')
print(f"Institutions loaded: {len(df_institutions):,}")

Programs loaded: 229,188
Institutions loaded: 6,429


In [ ]:
# Merge programs + institutions
merged_df = df_programs.merge(df_institutions, on='UNITID', how='left')
print(f"Merged dataset: {len(merged_df):,} rows, {merged_df.shape[1]} columns")

# Clean up memory
del df_programs, df_institutions
gc.collect()

In [ ]:
print("\nCleaning privacy-suppressed values...")

# Replace PS/PrivacySuppressed with NaN
merged_df = merged_df.replace(['PrivacySuppressed', 'PS', 'NULL', 'null'], np.nan)

print("Privacy-suppressed values cleaned up to NaN")


In [ ]:
merged_df.info()

In [ ]:
print("\nConverting columns to numeric...")

# Get numeric columns (exclude text columns)
numeric_cols = [col for col in merged_df.columns
                if col not in ['INSTNM', 'CIPDESC', 'CREDDESC', 'STABBR']]

# Convert
for col in numeric_cols:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')

print(f"Converted {len(numeric_cols)} columns to numeric")

In [ ]:
merged_df.info()

In [ ]:
merged_df_nops = merged_df.copy()  # "nops" = no privacy suppressed

print(f"\n Cleaned dataset ready: {merged_df_nops.shape}")
print(f"  Rows: {len(merged_df_nops):,}")
print(f"  Columns: {merged_df_nops.shape[1]}")

In [ ]:
print("\nData types:")
print(merged_df_nops.dtypes)

In [ ]:
print("\nMissing data summary:")
missing = merged_df_nops.isnull().sum()
missing_pct = (missing / len(merged_df_nops) * 100).round(1)

for col in merged_df_nops.columns:
    if missing[col] > 0:
        print(f"  {col:25s}: {missing[col]:,} ({missing_pct[col]}%)")

In [ ]:
column_definitions = {
    'UNITID': 'Institution ID',
    'INSTNM': 'Institution name',
    'CIPCODE': 'Major code',
    'CIPDESC': 'Major name',
    'CREDLEV': 'Credential level (1=Cert, 3=Bach, 5=Master, etc)',
    'CREDDESC': 'Credential description',
    'EARN_MDN_4YR': 'Median earnings 4 years after grad',
    'EARN_MDN_1YR': 'Median earnings 1 year after grad',
    'EARN_COUNT_WNE_4YR': 'Sample size for 4yr earnings',
    'EARN_COUNT_WNE_1YR': 'Sample size for 1yr earnings',
    'DISTANCE': 'Online program flag',
    'STABBR': 'State',
    'CONTROL': 'Institution type (1=Public, 2=Private, 3=For-profit)',
    'REGION': 'Geographic region',
    'ADM_RATE': 'Admission rate',
    'COSTT4_A': 'Annual cost',
    'TUITIONFEE_IN': 'Tuition',
    'PCTPELL': 'Percent receiving Pell grants (low-income proxy)',
    'C150_4': 'Graduation rate',
    'MD_EARN_WNE_P10': 'School-wide median earnings (10yr)'
}

print("\nColumn definitions:")
for col, definition in column_definitions.items():
    print(f"  {col:25s}: {definition}")

In [ ]:
has_4yr = merged_df_nops['EARN_MDN_4YR'].notna()
has_1yr = merged_df_nops['EARN_MDN_1YR'].notna()
has_both = has_4yr & has_1yr
has_either = has_4yr | has_1yr

print(f"\nTotal programs: {len(merged_df_nops):,}")
print(f"4-year earnings: {has_4yr.sum():,} ({has_4yr.sum()/len(merged_df_nops)*100:.1f}%)")
print(f"1-year earnings: {has_1yr.sum():,} ({has_1yr.sum()/len(merged_df_nops)*100:.1f}%)")
print(f"BOTH: {has_both.sum():,} ({has_both.sum()/len(merged_df_nops)*100:.1f}%)")
print(f"EITHER: {has_either.sum():,} ({has_either.sum()/len(merged_df_nops)*100:.1f}%)")

print(f"\n Imputation potential:")
print(f"  Without imputation: {has_4yr.sum():,} programs")
print(f"  With imputation: {has_either.sum():,} programs")
print(f"  Gain: +{has_either.sum() - has_4yr.sum():,}")

In [ ]:
# Get programs with BOTH 1yr and 4yr
df_both = merged_df_nops[has_both].copy()
print(f"\nPrograms with BOTH earnings: {len(df_both):,}")

In [ ]:
df_both['GROWTH_FACTOR'] = df_both['EARN_MDN_4YR'] / df_both['EARN_MDN_1YR']

print(f"\nGrowth factor stats:")
print(f"  Median: {df_both['GROWTH_FACTOR'].median():.3f}x")
print(f"  Mean: {df_both['GROWTH_FACTOR'].mean():.3f}x")
print(f"  Min: {df_both['GROWTH_FACTOR'].min():.3f}x")
print(f"  Max: {df_both['GROWTH_FACTOR'].max():.3f}x")

In [ ]:
print("\nRemoving outliers...")
print(f"Before: {len(df_both):,}")

# Keep reasonable range (0.5x to 3.0x)
reasonable = (df_both['GROWTH_FACTOR'] >= 0.5) & (df_both['GROWTH_FACTOR'] <= 3.0)
df_both = df_both[reasonable].copy()

print(f"After: {len(df_both):,}")
print(f"Removed: {(~reasonable).sum():,}")

print(f"\nCleaned median: {df_both['GROWTH_FACTOR'].median():.3f}x")

Why chose median growth factor?

In [ ]:
# Calculate by institution type
growth_by_control = df_both.groupby('CONTROL')['GROWTH_FACTOR'].agg(['median', 'mean', 'count'])
overall_growth = df_both['GROWTH_FACTOR'].median()

control_names = {1: 'Public', 2: 'Private Nonprofit', 3: 'For-Profit'}

print(f"\n{'Type':<22} {'Median':<10} {'Mean':<10} {'Count':<10}")
print("-" * 52)

for control in sorted(growth_by_control.index):
    if not np.isnan(control):
        name = control_names.get(control, f'Type {int(control)}')
        med = growth_by_control.loc[control, 'median']
        mn = growth_by_control.loc[control, 'mean']
        cnt = int(growth_by_control.loc[control, 'count'])
        print(f"{name:<22} {med:<10.3f} {mn:<10.3f} {cnt:<10,}")

print("-" * 52)
print(f"{'Overall':<22} {overall_growth:<10.3f} {df_both['GROWTH_FACTOR'].mean():<10.3f} {len(df_both):<10,}")

# Save for imputation
growth_factors = {
    'overall': overall_growth,
    'by_control': growth_by_control['median'].to_dict()
}

print("\nGrowth factors saved")

In [ ]:
# Quick visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution
axes[0].hist(df_both['GROWTH_FACTOR'], bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(overall_growth, color='red', linestyle='--', linewidth=2,
                label=f'Median: {overall_growth:.3f}x')
axes[0].set_xlabel('Growth Factor')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Growth Factor Distribution')
axes[0].legend()

# By institution type
growth_medians = growth_by_control['median'].sort_values(ascending=False)
axes[1].bar([control_names.get(c, str(int(c))) for c in growth_medians.index],
            growth_medians.values, color=['green', 'blue', 'orange'], alpha=0.7)
axes[1].set_ylabel('Median Growth Factor')
axes[1].set_title('Growth by Institution Type')
axes[1].axhline(overall_growth, color='red', linestyle='--', alpha=0.5)
for i, v in enumerate(growth_medians.values):
    axes[1].text(i, v + 0.01, f'{v:.3f}x', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('growth_factors.png', dpi=300)
print("Saved: growth_factors.png")
plt.show()

print orignial 1 year too to make sense

In [ ]:
# Create tracking columns
merged_df_nops['EARN_4YR_SOURCE'] = 'missing'
merged_df_nops['EARN_MDN_4YR_FINAL'] = merged_df_nops['EARN_MDN_4YR'].copy()

# Mark original 4-year data
has_4yr = merged_df_nops['EARN_MDN_4YR'].notna()
merged_df_nops.loc[has_4yr, 'EARN_4YR_SOURCE'] = 'original'

print(f"\nOriginal 4-year data: {has_4yr.sum():,}")

# Programs needing imputation (have 1yr but not 4yr)
needs_impute = (~has_4yr) & merged_df_nops['EARN_MDN_1YR'].notna()
print(f"Programs to impute: {needs_impute.sum():,}")

In [ ]:
imputed_count = 0

for idx in merged_df_nops[needs_impute].index:
    control_type = merged_df_nops.loc[idx, 'CONTROL']
    earn_1yr = merged_df_nops.loc[idx, 'EARN_MDN_1YR']

    # Use institution-specific growth factor
    if pd.notna(control_type) and control_type in growth_factors['by_control']:
        growth = growth_factors['by_control'][control_type]
    else:
        growth = growth_factors['overall']

    merged_df_nops.loc[idx, 'EARN_MDN_4YR_FINAL'] = earn_1yr * growth
    merged_df_nops.loc[idx, 'EARN_4YR_SOURCE'] = 'imputed'
    imputed_count += 1

    if imputed_count % 5000 == 0:
        print(f"  {imputed_count:,} / {needs_impute.sum():,}")

print(f"\nImputation complete: {imputed_count:,} programs")

In [ ]:
# Final summary
source_counts = merged_df_nops['EARN_4YR_SOURCE'].value_counts()

print(f"\nFinal earnings coverage:")
for source, count in source_counts.items():
    pct = count / len(merged_df_nops) * 100
    print(f"  {source.capitalize():10s}: {count:,} ({pct:.1f}%)")

total_usable = (merged_df_nops['EARN_4YR_SOURCE'] != 'missing').sum()
print(f"\nTotal usable: {total_usable:,} ({total_usable/len(merged_df_nops)*100:.1f}%)")

In [ ]:
has_4yr = merged_df_nops['EARN_MDN_4YR_FINAL'].notna()
print(f"\nOriginal 4-year data: {has_4yr.sum():,}")

In [ ]:
merged_df_nops.info()

In [ ]:
print("\n" + "="*70)
print("EDA: CORRELATION ANALYSIS")
print("="*70)

# Select numeric features
numeric_features = [
    'EARN_MDN_4YR', 'EARN_MDN_1YR', 'COSTT4_A', 'TUITIONFEE_IN',
    'ADM_RATE', 'C150_4', 'PCTPELL', 'MD_EARN_WNE_P10',
    'CREDLEV', 'CONTROL', 'REGION'
]

corr_matrix = merged_df_nops[numeric_features].corr()

print("\nHigh correlations (|r| > 0.7):")
high_corr = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > 0.7:
            high_corr.append({
                'Feature_1': corr_matrix.columns[i],
                'Feature_2': corr_matrix.columns[j],
                'Correlation': corr_matrix.iloc[i, j]
            })
            print(f"  {corr_matrix.columns[i]:20s} <-> {corr_matrix.columns[j]:20s}: r = {corr_matrix.iloc[i, j]:.3f}")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Heatmap
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8}, ax=axes[0])
axes[0].set_title('Feature Correlation Matrix', fontsize=14, fontweight='bold')

# Correlations with 4yr earnings
earnings_corr = corr_matrix['EARN_MDN_4YR'].drop('EARN_MDN_4YR').sort_values()
colors = ['red' if x < 0 else 'green' for x in earnings_corr.values]

axes[1].barh(range(len(earnings_corr)), earnings_corr.values, color=colors, alpha=0.7)
axes[1].set_yticks(range(len(earnings_corr)))
axes[1].set_yticklabels(earnings_corr.index)
axes[1].set_xlabel('Correlation with 4-Year Earnings')
axes[1].set_title('Feature Correlations with Earnings', fontsize=14, fontweight='bold')
axes[1].axvline(0, color='black', linewidth=1)

plt.tight_layout()
plt.savefig('eda_correlation.png', dpi=300)
print("\n✓ Saved: eda_correlation.png")
plt.show()

print("\nKey Finding: COSTT4_A vs TUITIONFEE_IN (r=0.98) → Will drop TUITIONFEE_IN")

In [ ]:
print("\n" + "="*70)
print("EDA: KEY VISUALIZATIONS")
print("="*70)

# Create 6-panel visualization
fig = plt.figure(figsize=(18, 12))

# 1. Earnings vs Cost scatter
ax1 = plt.subplot(2, 3, 1)
has_both_data = merged_df_nops['EARN_MDN_4YR'].notna() & merged_df_nops['COSTT4_A'].notna()
df_scatter = merged_df_nops[has_both_data].copy()

control_colors = {1.0: 'green', 2.0: 'blue', 3.0: 'orange'}
control_names = {1.0: 'Public', 2.0: 'Private', 3.0: 'For-Profit'}

for control, color in control_colors.items():
    mask = df_scatter['CONTROL'] == control
    ax1.scatter(df_scatter[mask]['COSTT4_A'], df_scatter[mask]['EARN_MDN_4YR'],
               alpha=0.3, s=5, c=color, label=control_names[control])
ax1.plot([0, 90000], [0, 90000], 'r--', linewidth=2, alpha=0.5, label='Break-even')
ax1.set_xlabel('Annual Cost ($)')
ax1.set_ylabel('4-Year Earnings ($)')
ax1.set_title('Earnings vs Cost by Institution Type')
ax1.legend(fontsize=8)
ax1.grid(alpha=0.3)

# 2. Earnings distribution
ax2 = plt.subplot(2, 3, 2)
earn_data = merged_df_nops['EARN_MDN_4YR'].dropna()
ax2.hist(earn_data, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
ax2.axvline(earn_data.median(), color='red', linestyle='--', linewidth=2,
           label=f'Median: ${earn_data.median():,.0f}')
ax2.set_xlabel('4-Year Earnings ($)')
ax2.set_ylabel('Frequency')
ax2.set_title('4-Year Earnings Distribution')
ax2.legend()

# 3. Cost distribution
ax3 = plt.subplot(2, 3, 3)
cost_data = merged_df_nops['COSTT4_A'].dropna()
ax3.hist(cost_data, bins=50, edgecolor='black', alpha=0.7, color='coral')
ax3.axvline(cost_data.median(), color='red', linestyle='--', linewidth=2,
           label=f'Median: ${cost_data.median():,.0f}')
ax3.set_xlabel('Annual Cost ($)')
ax3.set_ylabel('Frequency')
ax3.set_title('Annual Cost Distribution')
ax3.legend()

# 4. Programs by institution type
ax4 = plt.subplot(2, 3, 4)
control_counts = merged_df_nops['CONTROL'].value_counts().sort_index()
labels = ['Public', 'Private', 'For-Profit']
colors_bar = ['green', 'blue', 'orange']
ax4.bar(labels, control_counts.values, color=colors_bar, alpha=0.7)
ax4.set_ylabel('Number of Programs')
ax4.set_title('Programs by Institution Type')
for i, v in enumerate(control_counts.values):
    ax4.text(i, v + 2000, f'{v:,}', ha='center', fontweight='bold')

# 5. Programs by credential level
ax5 = plt.subplot(2, 3, 5)
cred_counts = merged_df_nops['CREDLEV'].value_counts().sort_index()
cred_labels = {1: 'Cert', 2: 'Assoc', 3: 'Bach', 4: 'PB', 5: 'Master', 6: 'Doc', 7: '1st Prof', 8: 'Grad'}
labels_cred = [cred_labels.get(int(x), str(x)) for x in cred_counts.index]
ax5.bar(labels_cred, cred_counts.values, color='purple', alpha=0.7)
ax5.set_ylabel('Number of Programs')
ax5.set_title('Programs by Credential Level')
ax5.tick_params(axis='x', rotation=45)

# 6. Top majors
ax6 = plt.subplot(2, 3, 6)
top_majors = merged_df_nops['CIPDESC'].value_counts().head(10)
ax6.barh(range(len(top_majors)), top_majors.values, color='teal', alpha=0.7)
ax6.set_yticks(range(len(top_majors)))
ax6.set_yticklabels([label[:35] + '...' if len(label) > 35 else label for label in top_majors.index], fontsize=8)
ax6.set_xlabel('Number of Programs')
ax6.set_title('Top 10 Most Common Majors')
ax6.invert_yaxis()

plt.tight_layout()
plt.savefig('eda_comprehensive.png', dpi=300, bbox_inches='tight')
print(" Saved: eda_comprehensive.png (6 plots)")
plt.show()

In [ ]:
print("\n" + "="*70)
print("PREPARE FOR TRAIN-TEST SPLIT")
print("="*70)

# Keep all programs with either earnings (no cost filter yet)
df = merged_df_nops.copy()
df_usable = df[(df['EARN_MDN_4YR'].notna()) | (df['EARN_MDN_1YR'].notna())].copy()

print(f"Original: {len(df):,}")
print(f"With earnings: {len(df_usable):,}")

# Breakdown
has_4yr = df_usable['EARN_MDN_4YR'].notna()
has_1yr = df_usable['EARN_MDN_1YR'].notna()
print(f"\n4-year only: {(has_4yr & ~has_1yr).sum():,}")
print(f"1-year only: {(~has_4yr & has_1yr).sum():,}")
print(f"Both: {(has_4yr & has_1yr).sum():,}")

# Train-test split
from sklearn.model_selection import train_test_split

unique_units = df_usable['UNITID'].unique()
train_units, test_units = train_test_split(unique_units, test_size=0.2, random_state=42)

df_train = df_usable[df_usable['UNITID'].isin(train_units)].copy()
df_test = df_usable[df_usable['UNITID'].isin(test_units)].copy()

print(f"\nTrain: {len(df_train):,} | Test: {len(df_test):,}")
print("Split BEFORE imputation (no leakage)")

In [ ]:
print("\n" + "="*70)
print("CALCULATE IMPUTATION STATS (TRAIN ONLY)")
print("="*70)

# Growth factors from train
train_both = df_train[df_train['EARN_MDN_1YR'].notna() & df_train['EARN_MDN_4YR'].notna()].copy()
train_both['GROWTH_FACTOR'] = train_both['EARN_MDN_4YR'] / train_both['EARN_MDN_1YR']
train_both = train_both[(train_both['GROWTH_FACTOR'] >= 0.5) & (train_both['GROWTH_FACTOR'] <= 3.0)].copy()

growth_factors = {
    'overall': train_both['GROWTH_FACTOR'].median(),
    'by_control': train_both.groupby('CONTROL')['GROWTH_FACTOR'].median().to_dict()
}

print(f"Growth factors from {len(train_both):,} train programs:")
for c, g in growth_factors['by_control'].items():
    print(f"  CONTROL {c}: {g:.3f}x")

# Institutional medians from train
inst_medians = {col: df_train.groupby('CONTROL')[col].median().to_dict()
                for col in ['ADM_RATE', 'C150_4', 'PCTPELL', 'MD_EARN_WNE_P10']}

print("Stats calculated from TRAIN only")

In [ ]:
print("\n" + "="*70)
print("APPLY IMPUTATION")
print("="*70)

def impute_set(df, growth_factors, inst_medians):
    df = df.copy()

    # Earnings imputation
    df['EARN_4YR_SOURCE'] = 'missing'
    df['EARN_MDN_4YR_FINAL'] = df['EARN_MDN_4YR'].copy()
    df.loc[df['EARN_MDN_4YR'].notna(), 'EARN_4YR_SOURCE'] = 'original'

    needs_impute = df['EARN_MDN_4YR'].isna() & df['EARN_MDN_1YR'].notna()
    for idx in df[needs_impute].index:
        control = df.loc[idx, 'CONTROL']
        growth = growth_factors['by_control'].get(control, growth_factors['overall']) if pd.notna(control) else growth_factors['overall']
        df.loc[idx, 'EARN_MDN_4YR_FINAL'] = df.loc[idx, 'EARN_MDN_1YR'] * growth
        df.loc[idx, 'EARN_4YR_SOURCE'] = 'imputed'

    # Institutional features
    for col, medians in inst_medians.items():
        for control, val in medians.items():
            df.loc[(df['CONTROL']==control) & df[col].isna(), col] = val

    df['SAMPLE_SIZE'] = df['EARN_COUNT_WNE_4YR'].fillna(df['EARN_COUNT_WNE_1YR'])
    return df

df_train = impute_set(df_train, growth_factors, inst_medians)
df_test = impute_set(df_test, growth_factors, inst_medians)

print(f"Train - Original: {(df_train['EARN_4YR_SOURCE']=='original').sum():,}, Imputed: {(df_train['EARN_4YR_SOURCE']=='imputed').sum():,}")
print(f"Test - Original: {(df_test['EARN_4YR_SOURCE']=='original').sum():,}, Imputed: {(df_test['EARN_4YR_SOURCE']=='imputed').sum():,}")
print("Imputation complete (no leakage)")

In [ ]:
print("\n" + "="*70)
print("APPLY QUALITY FILTERS")
print("="*70)

def apply_filters(df):
    return df[
        (df['EARN_MDN_4YR_FINAL'].notna()) &
        (df['EARN_MDN_4YR_FINAL'] > 10000) &
        (df['EARN_MDN_4YR_FINAL'] < 350000) &
        (df['COSTT4_A'].notna()) &
        (df['COSTT4_A'] > 10000) &
        (df['COSTT4_A'] < 400000) &
        (df['SAMPLE_SIZE'] >= 5)
    ].copy()

df_train = apply_filters(df_train)
df_test = apply_filters(df_test)

print(f"Train: {len(df_train):,} | Test: {len(df_test):,} | Total: {len(df_train)+len(df_test):,}")
print("Quality filters applied")

In [ ]:
print("\n" + "="*70)
print("FEATURE ENGINEERING")
print("="*70)

def create_features(df):
    df = df.copy()
    df['ROI_4YR'] = ((df['EARN_MDN_4YR_FINAL'] * 4 - df['COSTT4_A'] * 4) / (df['COSTT4_A'] * 4)) * 100
    df['EARN_PREMIUM'] = df['EARN_MDN_4YR_FINAL'] / df['MD_EARN_WNE_P10']
    return df

df_train = create_features(df_train)
df_test = create_features(df_test)

print(f"Created features: ROI_4YR, EARN_PREMIUM")
print(f"\nROI_4YR stats (train):")
print(df_train['ROI_4YR'].describe())

# Visualize target
plt.figure(figsize=(10, 6))
plt.hist(df_train['ROI_4YR'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
plt.axvline(df_train['ROI_4YR'].median(), color='red', linestyle='--', linewidth=2,
           label=f"Median: {df_train['ROI_4YR'].median():.1f}%")
plt.xlabel('ROI (%)')
plt.ylabel('Frequency')
plt.title('ROI Distribution (Regression Target)')
plt.legend()
plt.savefig('target_distribution.png', dpi=300)
print("\nSaved: target_distribution.png")
plt.show()

In [ ]:
print("\n" + "="*70)
print("DROP REDUNDANT COLUMNS")
print("="*70)

drop_cols = ['TUITIONFEE_IN', 'EARN_MDN_1YR', 'EARN_MDN_4YR', 'EARN_COUNT_WNE_1YR',
             'EARN_COUNT_WNE_4YR', 'CIPDESC', 'CREDDESC']

df_train = df_train.drop(columns=[c for c in drop_cols if c in df_train.columns])
df_test = df_test.drop(columns=[c for c in drop_cols if c in df_test.columns])

print(f"Remaining columns: {df_train.shape[1]}")
print("Dropped redundant features")

In [ ]:
print("\n" + "="*70)
print("ENCODE CATEGORICALS")
print("="*70)

target = 'ROI_4YR'

# One-hot encode (fit on train, apply to both)
small_cat = ['CREDLEV', 'CONTROL', 'REGION', 'DISTANCE', 'EARN_4YR_SOURCE']
df_train_enc = pd.get_dummies(df_train, columns=small_cat, drop_first=True, dtype=int)
df_test_enc = pd.get_dummies(df_test, columns=small_cat, drop_first=True, dtype=int)

# Align columns
df_train_enc, df_test_enc = df_train_enc.align(df_test_enc, join='left', axis=1, fill_value=0)

# Target encode (using TRAIN means)
cipcode_means = df_train.groupby('CIPCODE')[target].mean()
df_train_enc['CIPCODE_ENCODED'] = df_train['CIPCODE'].map(cipcode_means)
df_test_enc['CIPCODE_ENCODED'] = df_test['CIPCODE'].map(cipcode_means)

stabbr_means = df_train.groupby('STABBR')[target].mean()
df_train_enc['STABBR_ENCODED'] = df_train['STABBR'].map(stabbr_means)
df_test_enc['STABBR_ENCODED'] = df_test['STABBR'].map(stabbr_means)

df_train_enc = df_train_enc.drop(columns=['CIPCODE', 'STABBR'])
df_test_enc = df_test_enc.drop(columns=['CIPCODE', 'STABBR'])

print(f"Encoded shape: {df_train_enc.shape}")
print("Encoding complete")

In [ ]:
print("\n" + "="*70)
print("PREPARE FEATURES & TARGET")
print("="*70)

# Separate X and y
X_train = df_train_enc.drop(columns=[target, 'UNITID', 'INSTNM'])
y_train = df_train_enc[target]

X_test = df_test_enc.drop(columns=[target, 'UNITID', 'INSTNM'])
y_test = df_test_enc[target]

print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")

In [ ]:
print("\n" + "="*70)
print("SCALE FEATURES")
print("="*70)

from sklearn.preprocessing import StandardScaler

numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
scaler = StandardScaler()

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test_scaled[numeric_cols] = scaler.transform(X_test[numeric_cols])

print(f" Scaled {len(numeric_cols)} features")
print("Ready for modeling")

In [ ]:
print("\n" + "="*70)
print("STEP 1: TRAIN DEFAULT RANDOM FOREST")
print("="*70)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Default parameters
rf_default = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
    n_jobs=-1,
    verbose=0
)

print("Training with default parameters...")
print("  n_estimators: 100")
print("  max_depth: None (unlimited)")
print("  min_samples_split: 2")
print("  max_features: 'sqrt'")

rf_default.fit(X_train_scaled, y_train)

# Predictions
y_train_pred = rf_default.predict(X_train_scaled)
y_test_pred = rf_default.predict(X_test_scaled)

# Evaluation
print("\n" + "="*70)
print("DEFAULT MODEL PERFORMANCE")
print("="*70)

print("\nTrain Set:")
train_r2 = r2_score(y_train, y_train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_mae = mean_absolute_error(y_train, y_train_pred)
print(f"  R²:   {train_r2:.4f}")
print(f"  RMSE: {train_rmse:.2f}%")
print(f"  MAE:  {train_mae:.2f}%")

print("\nTest Set:")
test_r2 = r2_score(y_test, y_test_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_mae = mean_absolute_error(y_test, y_test_pred)
print(f"  R²:   {test_r2:.4f}")
print(f"  RMSE: {test_rmse:.2f}%")
print(f"  MAE:  {test_mae:.2f}%")

# Check overfitting
gap = train_r2 - test_r2
print(f"\nOverfitting Check:")
print(f"  Train R² - Test R²: {gap:.4f}")
if gap > 0.1:
    print("  Overfitting detected (gap > 0.1)")
    print("  Hyperparameter tuning needed")
else:
    print("  Reasonable generalization")

print("\nDefault Random Forest trained")

In [ ]:
print("\n" + "="*70)
print("CHECK FOR LEAKED FEATURES")
print("="*70)

# Print all features
print(f"\nTotal features: {X_train.shape[1]}")
print(f"\nFeature list:")
for i, col in enumerate(X_train.columns, 1):
    print(f"  {i:3d}. {col}")

# Check for suspicious features
suspicious = []
for col in X_train.columns:
    if 'EARN_MDN_4YR' in col:
        suspicious.append(col)
    if 'EARN_PREMIUM' in col:
        suspicious.append(col)
    if 'ROI' in col.upper():
        suspicious.append(col)

if suspicious:
    print(f"\nLEAKED FEATURES FOUND:")
    for feat in suspicious:
        print(f" {feat}")
else:
    print("\n No obvious leaked features")

In [ ]:
print("="*70)
print("CORRECTED FEATURE PREPARATION")
print("="*70)

# Define leaked features (contain target information)
leaked_features = [
    'EARN_MDN_4YR_FINAL',      # Directly calculates ROI
    'EARN_PREMIUM',             # Contains earnings
    'SAMPLE_SIZE',              # Not known before enrollment
    'EARN_4YR_SOURCE_original'  # Metadata only
]

print("Features to EXCLUDE (data leakage):")
for feat in leaked_features:
    print(f" {feat}")

# Separate X and y with correct features
X_train = df_train_enc.drop(columns=['ROI_4YR', 'UNITID', 'INSTNM'] +
                                     [f for f in leaked_features if f in df_train_enc.columns])
y_train = df_train_enc['ROI_4YR']

X_test = df_test_enc.drop(columns=['ROI_4YR', 'UNITID', 'INSTNM'] +
                                   [f for f in leaked_features if f in df_test_enc.columns])
y_test = df_test_enc['ROI_4YR']

print(f"\nFeature matrix prepared")
print(f"  X_train: {X_train.shape}")
print(f"  X_test: {X_test.shape}")
print(f"  Features: {X_train.shape[1]} (removed {len(leaked_features)} leaked)")

print("\nValid features (known before enrollment):")
for i, col in enumerate(X_train.columns[:10], 1):
    print(f"  {i}. {col}")
print(f"  ... and {X_train.shape[1] - 10} more")

In [ ]:
print("\n" + "="*70)
print("SCALE FEATURES")
print("="*70)

from sklearn.preprocessing import StandardScaler

numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
scaler = StandardScaler()

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test_scaled[numeric_cols] = scaler.transform(X_test[numeric_cols])

print(f"Scaled {len(numeric_cols)} numeric features")
print(" Scaler fit on TRAIN only")
print(" Ready for modeling (NO DATA LEAKAGE)")

In [ ]:
print("\n" + "="*70)
print("MODEL: RANDOM FOREST REGRESSOR (DEFAULT)")
print("="*70)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Train with default parameters
rf_default = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
    n_jobs=-1,
    verbose=0
)

print("Training Random Forest with default parameters...")
print("  n_estimators: 100")
print("  max_depth: None (unlimited)")
print("  min_samples_split: 2")
print("  max_features: 'sqrt'")

rf_default.fit(X_train_scaled, y_train)

# Predictions
y_train_pred = rf_default.predict(X_train_scaled)
y_test_pred = rf_default.predict(X_test_scaled)

# Evaluation
print("\n" + "="*70)
print("DEFAULT MODEL PERFORMANCE")
print("="*70)

print("\nTrain Set:")
train_r2 = r2_score(y_train, y_train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_mae = mean_absolute_error(y_train, y_train_pred)
print(f"  R²:   {train_r2:.4f}")
print(f"  RMSE: {train_rmse:.2f}%")
print(f"  MAE:  {train_mae:.2f}%")

print("\nTest Set:")
test_r2_default = r2_score(y_test, y_test_pred)
test_rmse_default = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_mae_default = mean_absolute_error(y_test, y_test_pred)
print(f"  R²:   {test_r2_default:.4f}")
print(f"  RMSE: {test_rmse_default:.2f}%")
print(f"  MAE:  {test_mae_default:.2f}%")

# Overfitting check
gap = train_r2 - test_r2_default
print(f"\nOverfitting Check:")
print(f"  Train R² - Test R²: {gap:.4f}")
if gap > 0.1:
    print("  Overfitting detected - hyperparameter tuning recommended")
elif gap < 0:
    print("  Underfitting - test performing better than train (unusual)")
else:
    print("  Reasonable generalization")

print("\nDefault Random Forest trained")
print(f"Baseline established: R² = {test_r2_default:.4f}")

In [ ]:
print("\n" + "="*70)
print("HYPERPARAMETER TUNING")
print("="*70)

from sklearn.model_selection import RandomizedSearchCV

print("""
Strategy:
─────────
- Method: RandomizedSearchCV (faster than GridSearch)
- Cross-validation: 5-fold on TRAIN set only
- Scoring: R² (maximize explained variance)
- Iterations: 20 random combinations

Parameters to tune:
───────────────────
1. n_estimators [100, 200, 300]
   More trees = better performance, diminishing returns after 200

2. max_depth [10, 20, 30, None]
   Controls tree depth, prevents overfitting

3. min_samples_split [2, 5, 10]
   Minimum samples to split node, regularization

4. max_features ['sqrt', 'log2', 0.5]
   Features per split, controls randomness

5. min_samples_leaf [1, 2, 4]
   Minimum samples in leaf, smooths predictions
""")

# Parameter distribution
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'max_features': ['sqrt', 'log2', 0.5],
    'min_samples_leaf': [1, 2, 4]
}

print(f"Total combinations: {3*4*3*3*3} = 324")
print(f"Testing: 20 random combinations")
print(f"Total model fits: 20 × 5 = 100")

# RandomizedSearchCV
rf_random = RandomizedSearchCV(
    RandomForestRegressor(random_state=42, n_jobs=-1),
    param_distributions=param_dist,
    n_iter=20,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=2,
    random_state=42,
    return_train_score=True
)

print("\n" + "="*70)
print("TRAINING (5-10 minutes)")
print("="*70)

import time
start_time = time.time()

rf_random.fit(X_train_scaled, y_train)

elapsed_time = time.time() - start_time
print(f"\nTuning complete in {elapsed_time/60:.1f} minutes")


HYPERPARAMETER TUNING

Strategy:
─────────
- Method: RandomizedSearchCV (faster than GridSearch)
- Cross-validation: 5-fold on TRAIN set only
- Scoring: R² (maximize explained variance)
- Iterations: 20 random combinations

Parameters to tune:
───────────────────
1. n_estimators [100, 200, 300]
   More trees = better performance, diminishing returns after 200

2. max_depth [10, 20, 30, None]
   Controls tree depth, prevents overfitting

3. min_samples_split [2, 5, 10]
   Minimum samples to split node, regularization

4. max_features ['sqrt', 'log2', 0.5]
   Features per split, controls randomness

5. min_samples_leaf [1, 2, 4]
   Minimum samples in leaf, smooths predictions

Total combinations: 324 = 324
Testing: 20 random combinations
Total model fits: 20 × 5 = 100

TRAINING (5-10 minutes)
Fitting 5 folds for each of 20 candidates, totalling 100 fits


/usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: 